In [ ]:
import torch, mmengine, mmcv, mmdet, mmdet3d, spconv
print(torch.__version__)
print(torch.version.cuda)
print(mmengine.__version__)
print(mmcv.__version__)
print(mmdet.__version__)
print(mmdet3d.__version__)
print(spconv.__version__)

In [ ]:
import argparse
import logging
import os
import os.path as osp
import torch
import mmcv
import numpy as np

from mmengine.config import Config, DictAction
from mmengine.logging import print_log
from mmengine.registry import RUNNERS
from mmengine.runner import Runner
from mmengine import fileio

from mmdet3d.utils import replace_ceph_backend

from projects.mmdet3d_plugin.models.detectors import CmtDetector
import time
from mmengine.structures import InstanceData

In [ ]:
cfg = Config.fromfile('projects/configs/fusion/my_cmt_kitti.py')
# cfg = Config.fromfile('../mmdetection3d/projects/BEVFusion/configs/my_bevfusion.py')
cfg.work_dir = osp.abspath('./work_dirs')
runner = Runner.from_cfg(cfg)

In [ ]:
runner.model.train()

In [ ]:
with torch.no_grad():
    data_batch = next(iter(runner.train_dataloader))
    data_batch = runner.model.data_preprocessor(data_batch, training=False)
    batch_inputs_dict = data_batch['inputs']
    batch_data_samples = data_batch['data_samples']
    imgs = batch_inputs_dict.get('imgs', None)
    points = batch_inputs_dict.get('points', None)
    img_metas = [item.metainfo for item in batch_data_samples]
    gt_bboxes_3d = [item.get('gt_instances_3d')['bboxes_3d'] for item in batch_data_samples]
    gt_labels_3d = [item.get('gt_instances_3d')['labels_3d'] for item in batch_data_samples]

    # img_feats = runner.model.extract_img_feat(imgs, img_metas)
    # voxels, num_points, coors = runner.model.voxelize(points)
    # voxel_features = runner.model.pts_voxel_encoder(voxels, num_points, coors)
    # batch_size = coors[-1, 0] + 1
    # x1 = runner.model.pts_middle_encoder(voxel_features, coors, batch_size)
    # x2 = runner.model.pts_backbone(x1)
    # if runner.model.with_pts_neck:
    #     x3 = runner.model.pts_neck(x2)

In [ ]:
img = data_batch['inputs']['imgs'][0][0].permute(1, 2, 0).cpu().numpy()
img = mmcv.imdenormalize(img, mean=np.array([103.530, 116.280, 123.675]), std=np.array([57.375, 57.120, 58.395]), to_bgr=False)
img = img/255

In [ ]:
filename = img_metas[0].get('filename')
img_bytes = fileio.get(filename, backend_args=None)
img = mmcv.imfrombytes(img_bytes, flag='color', backend='cv2')
img = img.astype(np.float32)/255

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img)

In [ ]:
gt_bboxes_3d[0]

In [ ]:
img_feats[0].shape, img_feats[1].shape

In [ ]:
voxel_features.shape, x1.shape, x2[0].shape, x2[1].shape, x3[0].shape

In [ ]:
# 训练过程
for data_batch in runner.train_dataloader:
    data_batch = runner.model.data_preprocessor(data_batch, training=True)
    if isinstance(data_batch, dict):
        losses = runner.model(**data_batch, mode='loss')
    elif isinstance(data_batch, (list, tuple)):
        losses = runner.model(*data_batch, mode='loss')
    else:
        raise TypeError()
    break

In [ ]:
losses

In [ ]:
data_batch['inputs'], data_batch['data_samples'][0]

In [ ]:
runner.model.eval()

In [ ]:
# 验证过程
for data_batch in runner.val_dataloader:
    data_batch = runner.model.data_preprocessor(data_batch, training=False)
    if isinstance(data_batch, dict):
        outputs = runner.model(**data_batch, mode='predict')
    elif isinstance(data_batch, (list, tuple)):
        outputs = runner.model(**data_batch, mode='predict')
    else:
        raise TypeError()
    runner.val_evaluator.process(data_samples=outputs, data_batch=data_batch)
    break
# with torch.no_grad():
#     metrics = runner.val_evaluator.evaluate(len(runner.val_dataloader.dataset))

In [ ]:
data_batch['inputs']['imgs'][0][:,300:400,300:400]

In [ ]:
outputs[0]

In [ ]:
runner.val_dataloader

In [ ]:
time_start = time.time()
num = 0
with torch.no_grad():
    for data_batch in runner.val_dataloader:
        data_batch = runner.model.data_preprocessor(data_batch, training=False)
        if isinstance(data_batch, dict):
            outputs = runner.model(**data_batch, mode='predict')
        elif isinstance(data_batch, (list, tuple)):
            outputs = runner.model(**data_batch, mode='predict')
        else:
            raise TypeError()
        num += 1
        if num == 100:
            break
print(time.time() - time_start)

In [ ]:
time_start = time.time()
with torch.no_grad():
    data_batch_raw = next(iter(runner.val_dataloader))
    for _ in range(100):
        data_batch = runner.model.data_preprocessor(data_batch_raw, training=False)
        if isinstance(data_batch, dict):
            outputs = runner.model(**data_batch, mode='predict')
        elif isinstance(data_batch, (list, tuple)):
            outputs = runner.model(**data_batch, mode='predict')
        else:
            raise TypeError()
print(time.time() - time_start)

In [ ]:
with torch.no_grad():
    data_batch = next(iter(runner.val_dataloader))
    data_batch = runner.model.data_preprocessor(data_batch, training=False)
    batch_inputs_dict = data_batch['inputs']
    batch_data_samples = data_batch['data_samples']
    imgs = batch_inputs_dict.get('imgs', None)
    points = batch_inputs_dict.get('points', None)
    img_metas = [item.metainfo for item in batch_data_samples]

    time_start = time.time()
    for _ in range(100):
        img_feats = runner.model.extract_img_feat(imgs, img_metas)
    
    midedle1 = time.time()
    for _ in range(100):
        pts_feats = runner.model.extract_pts_feat(points, img_feats, img_metas)

    midedle2 = time.time()
    for _ in range(100):
        outs = runner.model.pts_bbox_head(pts_feats, img_feats, img_metas)

    middle3 = time.time()
    for _ in range(100):
        bbox_list = runner.model.pts_bbox_head.get_bboxes(
            outs, img_metas, rescale=False)
        
        # bbox_results = []
        # for bboxes, scores, labels in bbox_list:
        #     results = InstanceData()
        #     results.bboxes_3d = bboxes.to('cpu')
        #     results.scores_3d = scores.cpu()
        #     results.labels_3d = labels.cpu()
        #     bbox_results.append(results)
        # detsamples = runner.model.add_pred_to_datasample(batch_data_samples,
        #                                             data_instances_3d = bbox_results,
        #                                             data_instances_2d = None)
    end = time.time()
    print(midedle1 - time_start, midedle2 - midedle1, middle3 - midedle2, end - middle3)